In [ ]:
from itertools import product
import numpy as np
import pandas as pd

import chaospy

import matplotlib.pyplot as plt
plt.style.use("bmh")
%matplotlib inline

In [ ]:
factors = dict(
    solar=[0.75,1.0,1.25],
    onwind=[0.75,1.0,1.25],
    offwind=[0.75,1.0,1.25],
    H2=[0.75,1.0,1.25],  
    battery=[0.75,1.0,1.25]
)

In [ ]:
factors = dict(
    solar=[0.5,0.75,1.0,1.25,1.5],
    onwind=[0.5,0.75,1.0,1.25,1.5],
    offwind=[0.5,0.75,1.0,1.25,1.5],
    H2=[0.5,0.75,1.0,1.25,1.5],  
    battery=[0.5,0.75,1.0,1.25,1.5]
)

In [ ]:
opts = ["3H"]

In [ ]:
def add_cost_opts(wildcards_opts):
    carrier, values = zip(*factors.items())
    cost_sets = [dict(zip(carrier, v)) for v in product(*values)]

    new_opts = []
    for opts in wildcards_opts:
        for cost_set in cost_sets:
            cost_opts = "-".join([f"{c}+{v}" for c, v in cost_set.items()])
            new_opts.append(f"{opts}-{cost_opts}")
    
    return new_opts

In [ ]:
add_cost_opts(opts)

## Create `opts` from quasi-random samples

In [ ]:
# TODO: unify with add_cost_opts
def add_sampled_cost_opts(wildcards_opts, order=10):
    
    dist = {}
    dist["onwind"] = chaospy.Uniform(0.5, 1.5)
    dist["offwind"] = chaospy.Uniform(0.5,1.5)
    dist["solar"] = chaospy.Uniform(0.5, 1.5)
    dist["H2"] = chaospy.Uniform(0.5, 1.5)

    dist["battery"] = chaospy.Uniform(0.5,1.5)

    distribution = chaospy.J(*dist.values())
    
    samples = distribution.sample(order, rule="halton").round(3)
    
    new_opts = []
    for opts in wildcards_opts:
        for s in samples.T:
            cost_set = dict(zip(dist.keys(), s))
            cost_opts = "-".join([f"{c}+{v}
            " for c, v in cost_set.items()])
            new_opts.append(f"{opts}-{cost_opts}")
    
    return new_opts

In [ ]:
add_sampled_cost_opts(opts, 1000)

## Comparison of Low-Discrepancy Sampling Methods

In [ ]:
fig, axes = plt.subplots(2,4,figsize=(10,5))
for i, rule in enumerate(['random', 'halton', "latin", 'sobol', 'grid', 'korobov', 'chebyshev', 'hammersley']):
    ax = axes[int(i/4),i%4]
    order = 100
    dim = 2
    if rule == "latin":
        xs = lhs(order=order, dim=dim)
    else:
        if rule in ['grid', 'chebyshev']:
            order = order**(1/dim)
        xs = generate_samples(order=order, domain=dim, rule=rule)
    ax.scatter(*xs, marker='.', c='navy')
    ax.set_title(rule.capitalize())
    ax.set_ylim([0,1])
    ax.set_xlim([0,1])
    ax.set_xticks(np.arange(0,1.01,0.2))
    ax.set_yticks(np.arange(0,1.01,0.2))
plt.tight_layout()
plt.savefig("low-discrepancy-series.pdf", bbox_inches='tight')

## Multi-fidelity approach to build surrogate

In [ ]:
# TODO